<a href="https://colab.research.google.com/github/Diego15s/DIO-Baires-01-Transfer-learning/blob/main/Transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow keras matplotlib numpy

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import files

In [3]:
import tensorflow_datasets as tfds

dataset_name = "cats_vs_dogs"
(train_ds, test_ds), dataset_info = tfds.load(dataset_name, split=['train[:80%]', 'train[80%:]'], as_supervised=True, with_info=True)

In [4]:
base_model = keras.applications.MobileNetV2(input_shape=(160, 160, 3), include_top=False, weights="imagenet")
base_model.trainable = False  # Congela os pesos do modelo pré-treinado

In [5]:
model = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation='sigmoid')  # Saída binária: 0 ou 1 (Gato ou Cachorro)
])

In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
def resize_image(image, label):
    image = tf.image.resize_with_pad(image, 160, 160)  # Resize with padding to ensure consistent output shape
    return image, label

train_ds = train_ds.map(resize_image)
test_ds = test_ds.map(resize_image) # Apply resizing to the test dataset as well

history = model.fit(train_ds.batch(32), validation_data=test_ds.batch(32), epochs=5)

Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 524s 865ms/step - accuracy: 0.6947 - loss: 0.5929 - val_accuracy: 0.7519 - val_loss: 0.5058
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 483s 829ms/step - accuracy: 0.7711 - loss: 0.4769 - val_accuracy: 0.7595 - val_loss: 0.4921
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 482s 828ms/step - accuracy: 0.7872 - loss: 0.4514 - val_accuracy: 0.7661 - val_loss: 0.4896
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 496s 817ms/step - accuracy: 0.7932 - loss: 0.4421 - val_accuracy: 0.7642 - val_loss: 0.4910
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 532s 913ms/step - accuracy: 0.7997 - loss: 0.4292 - val_accuracy: 0.7657 - val_loss: 0.4921


In [9]:
loss, accuracy = model.evaluate(test_ds.batch(32))
print(f"Acurácia: {accuracy:.2f}")

146/146 ━━━━━━━━━━━━━━━━━━━━ 95s 647ms/step - accuracy: 0.7586 - loss: 0.4981
Acurácia: 0.77


In [10]:
# Fazer upload da imagem manualmente no Google Colab
uploaded = files.upload()

# Obter o nome do arquivo enviado
image_path = list(uploaded.keys())[0]

def predict_image(image_path):
    # Verificar se o arquivo foi realmente carregado
    if not os.path.exists(image_path):
        print(f"Erro: O arquivo '{image_path}' não foi encontrado.")
        return

          # Carregar e preparar a imagem
    img = keras.preprocessing.image.load_img(image_path, target_size=(160, 160))
    img_array = keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Fazer a previsão com o modelo
    prediction = model.predict(img_array)

    # Exibir o resultado
    if prediction[0] > 0.5:
        print("É um Cachorro! 🐶")
        print(f"Probabilidade de ser um Cachorro: {prediction[0]}")
        print(f"Probabilidade de ser um Gato: {1 - prediction[0]}")
    else:
        print("É um Gato! 🐱")
        print(f"Probabilidade de ser um Cachorro: {prediction[0]}")
        print(f"Probabilidade de ser um Gato: {1 - prediction[0]}")

# Chamar a função para prever a imagem carregada
predict_image(image_path)

Saving cat.jpg to cat.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
É um Gato! 🐱
Probabilidade de ser um Cachorro: [0.23603252]
Probabilidade de ser um Gato: [0.7639675]
